# Derivados Financieros 2023 [previa a la clase 3]

## 1) Modulo yfinance para Yahoo Finance
## 2) Plots
## 3) Payoffs - Estraegias


In [ ]:
pip install yfinance

In [ ]:
### Importo los modulos
import sys
sys.path.append('..')
from Codigo.data_nyse import *

import yfinance as yf
import datetime

## API Yahoo finance


Modulos necesarios

data_nyse (en webcampus) ;

pandas,

matplotlib,

datetime,

yfinance, (yfinance-0.1.63 - tuve que actualizar reicentemente)

sys,

Instalarlos con Anaconda, comando Conda o comando pip install


urllib.requestatexit
datetime
dateutil

import tda
json
selenium, webdriver


## Queremos emular lo que hicimos para BYMA

In [ ]:
### Metodo para el web scraping
panel_yahoo_GGAL = obtener_opciones_yahoo_finance('GGAL')
panel_yahoo_GGAL


In [ ]:

panel_ociones_GGAL_nyse_clean = obtener_panel_opciones_nyse('GGAL', True)
panel_ociones_GGAL_nyse_clean

## 1) Modulo yfinance para Yahoo Finance

In [ ]:
ticker = 'GGAL'
data = yf.Ticker(ticker)

#Data es un objeto
# data.info contiene informacion (mucha) del activo

data.info



In [ ]:
# Spot price

hist = data.history()

spot = hist.tail(1)['Close'].iloc[0]
spot

In [ ]:
# Vencimientos de opciones
vencimientos = data.options

vencimientos

In [ ]:
# Calls y puts
calls = data.option_chain(vencimientos[0])[0]
puts = data.option_chain(vencimientos[0])[1]

puts

### Que hace internamente el metodo??

In [ ]:
#Recorro cada vencimiento disponible y voy tomando calls por un lado y puts por otro
for vencimiento in vencimientos[1:]:
        calls = calls.append(data.option_chain(vencimiento)[0])
        puts = puts.append(data.option_chain(vencimiento)[1])

#Uno los puts y los calls
panel_opciones = calls.append(puts)

# Le agrego dos columnas utiles - Ticker y Spot

panel_opciones['Spot'] = hist.tail(1)['Close'].iloc[0]
panel_opciones['Ticker'] = ticker

panel_opciones.head(10)

## Del panel de Yahoo crudo a otro con informacion customizada (igual que antes)

#### *) Agrego columnas utiles

In [ ]:

panel_opciones['Moneyness'] = 0.0
panel_opciones['TTM'] = 0
panel_opciones['CallPut'] = ''
panel_opciones = panel_opciones.reset_index()

panel_opciones.head(5)

#### *) Recorro las filas (1ra como ejemplo)

In [ ]:
idx = panel_opciones.index.values[0]

In [ ]:
callput = panel_opciones.contractSymbol.values[idx][10:11]
panel_opciones['CallPut'].values[idx] = callput

In [ ]:
year = 2000 + int(panel_opciones.contractSymbol.values[idx][4:6])
month = int(panel_opciones.contractSymbol.values[idx][6:8])
day = int(panel_opciones.contractSymbol.values[idx][8:10])

hoy = datetime.date.today()
expiry_datetime = datetime.date(year, month, day)
ttm = (expiry_datetime - hoy).days

panel_opciones['TTM'].values[idx] = ttm

In [ ]:
panel_opciones['Moneyness'].values[idx] = panel_opciones['Spot'].values[idx] / panel_opciones['strike'].values[idx]

In [ ]:
panel_opciones.head(2)

#### *) Recorro las filas (las restantes como ejemplo)

In [ ]:
for idx in list(panel_opciones.index.values[1:]):
        year = 2000 + int(panel_opciones.contractSymbol.values[idx][4:6])
        month = int(panel_opciones.contractSymbol.values[idx][6:8])
        day = int(panel_opciones.contractSymbol.values[idx][8:10])
        
        hoy = datetime.date.today()
        expiry_datetime = datetime.date(year, month, day)
        ttm = (expiry_datetime - hoy).days

        panel_opciones['TTM'].values[idx] = ttm
        callput = panel_opciones.contractSymbol.values[idx][10:11]
        panel_opciones['CallPut'].values[idx] = callput
        
panel_opciones.Moneyness = panel_opciones.Spot/ panel_opciones.strike


In [ ]:
panel_opciones.head(10)

#### Panel Limpio

In [ ]:
panel_opciones = panel_opciones.rename(
            columns={'lastTradeDate': 'Last_Date', 'contractSymbol': 'Especie', 'strike': 'Strike', 'bid': 'Bid',
                     'ask': 'Ask', 'lastPrice': 'Last'}, inplace=False)

panel_opciones = panel_opciones.sort_values(['TTM', 'Ticker', 'Strike', 'CallPut'])

panel_opciones = panel_opciones[['Especie', 'Ticker', 'Spot', 'CallPut', 'Strike', 'TTM', 'Last', 'Bid','Ask','impliedVolatility','Moneyness']]


In [ ]:
panel_opciones = panel_opciones.reset_index()
del panel_opciones['index']
panel_opciones

## 2) Ploteando los datos

### Para cada vencimiento, grafico los strikes

In [ ]:
# NYSE
panel_opciones_ggal_nyse_limpio = obtener_panel_opciones_nyse('GGAL', True)

TTMs = list(set(panel_opciones_ggal_nyse_limpio.TTM.values))
TTMs

In [ ]:
TTM = 8
panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == "C")].head(10)

In [ ]:
from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs:
    Ks = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Last.values
    ## Last VS (B+A)/2
    plt.plot(Ks, precios,'*', label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:

CP = "P"
for TTM in TTMs:
    Ks = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios,'*', label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()

### Para cada strike, grafico los TTM

In [ ]:
Ks = list(set(panel_opciones_ggal_nyse_limpio.Strike.values))

In [ ]:
from matplotlib import pyplot as plt
CP = "C"
for K in Ks:
    TTMs = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.Strike == K) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].TTM.values
    precios = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.Strike == K) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Last.values
    if len(TTMs)>1:
        plt.plot(TTMs, precios,'*', label='Strike = '+str(K))

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al TTM')
    
plt.show()
    

## Ploteando los datos con un ticker mega liquido

In [ ]:
# NYSE AAPL
ticker = 'TSLA'
panel_opciones_nyse_limpio = obtener_panel_opciones_nyse(ticker, True)

TTMs = list(set(panel_opciones_nyse_limpio.TTM.values))
TTMs.sort()
TTMs

In [ ]:

from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs[2:5]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:

from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs[8:11]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:

from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs[2:5]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios,'.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for TTM in TTMs[3:6]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al Strike')
    
plt.show()
    

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for TTM in TTMs[10:15]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al Strike')
    
plt.show()
    

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for TTM in TTMs[11:17]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al Strike')
    
plt.show()
    

In [ ]:
Ks = list(set(panel_opciones_nyse_limpio.Strike.values))
Ks.sort()
len(Ks)

In [ ]:
from matplotlib import pyplot as plt
CP = "C"
for K in Ks[45:55]:
    TTMs = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].TTM.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    if len(TTMs)>1:
        plt.plot(TTMs, precios,'*', label='Strike = '+str(K))

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al TTM')
    
plt.show()

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for K in Ks[100:]:
    TTMs = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    if len(TTMs)>1:
        plt.plot(TTMs, precios,'*', label='Strike = '+str(K))

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al TTM')
    
plt.show()

## Pasar a Excel

In [ ]:
panel_opciones_ggal_nyse_crudo = obtener_panel_opciones_nyse('GGAL', False)
panel_opciones_ggal_nyse_crudo.to_excel('Clase_2_panel_opciones_ggal_nyse_crudo.xls')

panel_opciones_ggal_nyse_limpio = obtener_panel_opciones_nyse('GGAL', True)
panel_opciones_ggal_nyse_limpio.to_excel('Clase_2_panel_opciones_ggal_nyse_limpio.xls')


panel_opciones_aapl_nyse_limpio = obtener_panel_opciones_nyse('AAPL', True)
panel_opciones_aapl_nyse_limpio.to_excel('Clase_2_panel_opciones_aapl_nyse_limpio.xls')                                                
         

## 3) Payoffs y Estrategias

In [ ]:
import numpy as np
from matplotlib import pyplot as plt


### Vanilla Call y Put

In [ ]:
def payoff_call(S,K):
    return np.maximum(S-K,0)

def payoff_put(S,K):
    return np.maximum(K-S,0)


In [ ]:
K=50
S = np.linspace(0,100,50)


In [ ]:
#Grafico la call

plt.plot(S,payoff_call(S, K))
plt.show()


In [ ]:
#Grafico la put
plt.plot(S,payoff_put(S, K))
plt.show()

### Forward sintetico y Combo

In [ ]:
def payoff_forwardSintetico(S,K):
    return payoff_call(S,K)-payoff_put(S, K)

def payoff_combo(S,K1,K2):
    return payoff_call(S,K2)-payoff_put(S, K1)


In [ ]:
#Grafico el forward sintetico
plt.plot(S,payoff_forwardSintetico(S, K))
plt.show()



In [ ]:
K1 = 35
K2 = 65

In [ ]:
#Grafico el combo
plt.plot(S,payoff_combo(S, K1,K2))
plt.show()

### Spreads

In [ ]:
def payoff_BullCS(S,K1,K2):
    return payoff_call(S,K1)-payoff_call(S, K2)

def payoff_BearCS(S,K1,K2):
    return payoff_call(S,K2)-payoff_call(S, K1)

def payoff_BullPS(S,K1,K2):
    return payoff_put(S,K1)-payoff_put(S, K2)


def payoff_BearPS(S,K1,K2):
    return payoff_put(S,K2)-payoff_put(S, K1)


In [ ]:
#Grafico el Bull Call Spread
plt.plot(S,payoff_BullCS(S, K1, K2))
plt.show()


In [ ]:
#Grafico el Bear Call Spread
plt.plot(S,payoff_BearCS(S, K1, K2))
plt.show()


In [ ]:
#Grafico el Bull Put Spread
plt.plot(S,payoff_BullPS(S, K1, K2))
plt.show()



In [ ]:
#Grafico el Bear Put Spread
plt.plot(S,payoff_BearPS(S, K1, K2))
plt.show()



### Butterfly spreads

In [ ]:
def payoff_CButterflyS(S,K1,K2,K3):
    return payoff_call(S,K1)-2*payoff_call(S, K2)+payoff_call(S, K3)

def payoff_PButterflyS(S,K1,K2,K3):
    return payoff_put(S,K1)-2*payoff_put(S, K2)+payoff_put(S, K3)



In [ ]:
K1=35
K2=50
K3=65


In [ ]:
#Grafico el Call Buterfly Spread
plt.plot(S,payoff_CButterflyS(S, K1, K2,K3))
plt.show()




In [ ]:
#Grafico el Put Buterfly Spread
plt.plot(S,payoff_PButterflyS(S, K1, K2,K3))
plt.show()


### Straddle y Strangle

In [ ]:
def payoff_straddle(S,K):
    return payoff_call(S,K)+payoff_put(S, K)


def payoff_strangle(S,K1,K2):
    return payoff_call(S,K1)+payoff_put(S, K2)





In [ ]:

K1 = 35
K2 = 65


In [ ]:
#Grafico el Strangle
plt.plot(S,payoff_strangle(S, K1,K2))
plt.show()


In [ ]:
#Grafico el Straddle

plt.plot(S,payoff_straddle(S, K))
plt.show()


### Estrategia AD hoc - Iron Condor

In [ ]:
def payoff_iron_condor(S,K1,K2,K3,K4):
    return payoff_put(S,K1)-payoff_put(S, K2)-payoff_call(S,K3)+payoff_call(S,K4)

In [ ]:
K1 = 30
K2 = 45
K3 = 55
K4 = 70

In [ ]:
plt.plot(S,payoff_iron_condor(S, K1,K2,K3,K4))
plt.show()
